## Procédure d'installation du script sur anaconda

Installation de la librairie pymongo  
$ conda install -c anaconda pymongo

## Script de détermination influenceur / non influenceur

In [1]:
# Intégration de l'ensemble des éléments indispensable au bon fonctionnement du script
from pymongo import MongoClient
import pandas as pd
import numpy as np
import csv
import sys
from os import rename
import requests
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [2]:
## Création filenameCoprusInfluenceurs
def creationCorpusInfluenceurs(limit):
    # Connexion à la base de donnée Mongodb d'Influenzzz.
    client = MongoClient('mongodb://nusa.influenzzz.fr:27017')
    db = client["influencers"]
    db.authenticate("esieestudents", "esieeparisinfluenzzz2017")

    # On récupère la collection "website".
    collection = db["website"]

    # Récupération des corpus des sites
    cursor = collection.find({}, { "_id":0, "fulltext":1}).limit(limit)
    fullcorpusInfluenceur = pd.DataFrame(list(cursor))

    # Enregistrement CSV influenceurs
    fullcorpusInfluenceur.to_csv('corpusInfluenceurs.csv')

In [3]:
# Fonction de suppression des balises HTML
def cleanhtml(html_doc):
    soup = BeautifulSoup(html_doc, 'html5lib')
    
    for script in soup(["script", "style"]):
        script.extract()

    return soup.get_text()

In [4]:
# listURL LIST Liste des liens URL à tester ou récupérer
# testConnexion BOOLEAN True retourn liste de connexion valide, False pour un liste des corpus des sites valides

def connexionRecuperationCorpus(listURL, testConnexion):
    listCorpus = pd.DataFrame(columns=['fulltext'])
    for i, url in enumerate(listURL.values):
            # Session de connexion aux addresses URL
            MAX_RETRIES = 5
            session = requests.Session()
            adapter = requests.adapters.HTTPAdapter(max_retries=MAX_RETRIES)
            session.mount('https://', adapter)
            session.mount('http://', adapter)
            headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}

            try:
                with session as s:
                    res = s.get('http://'+url, headers=headers, timeout=2)
                    if res == None:
                        raise Exception
            except Exception as e:
                #print('An exception has been raised for : ' + url)
                pass
            else:
                listCorpus.loc[i] = url if testConnexion else cleanhtml(res.text)
    
    
    return listCorpus

In [5]:
### Instructions :
# Dans le dossier racine ne laisser fichier 'top-1m.csv' qu'à l'initialisation après une première initialistion
# et à l'exception d'un code erreur : 001, le fichier le dois pas être ajouter en fichier source.
# Seul le 'top-1m-own.csv' est suffisant au bon fonctionnement.

def creationCorpusNonInfluenceurs(dataCount):
    # Récupération de toutes les adresses URL de : top-1m.csv correpondant aux non influenceurs
    top1mFile = 'top-1m.csv'
    top1mFileOld = top1mFile + '.old'
    newTop1mFile = 'top-1m-own.csv'
    corpusFile = 'textSite.csv'

    while 1:
        try: open(top1mFile, 'r')
        except IOError or OSError:
            init = False
            file = newTop1mFile     
        else:
            init = True
            file = top1mFile

        # Récupération des liens URL du top1m
        listURL = pd.read_csv(file)
        listURL = listURL.iloc[:,-1]
        # Permet de réduire l'analyse
        if dataCount:
            listURL = listURL.iloc[:dataCount]

        testConnexion = True if init else False

        # Récupération de la dataframe contenant le corpus ou les liens URL si initialisation
        result = connexionRecuperationCorpus(listURL, testConnexion)

        # Affection aux différentes variables en fonction de l'initialisation ou non
        if init: noInfluenceur = result
        else: 
            fullcorpusNoInfluenceur = result

        if init:
            # Si ensemble site non influenceur assez grand, enregistrement dans un nouveau csv de tous les liens crawlable
            if len(noInfluenceur) < 5 and dataCount != None:
                print("Nombre de site non influenceur trop faible !\r\nCode erreur : 001")
            else:
                noInfluenceur.to_csv(newTop1mFile)

            # Renomme le fichier top1m pour bloquer l'initialisation
            rename(top1mFile, top1mFileOld)
        else:
            # Enregistrement CSV non influenceurs
            fullcorpusNoInfluenceur.to_csv('corpusNonInfluenceurs.csv')

            break

In [6]:
filenameCoprusInfluenceurs = 'corpusInfluenceurs.csv'
filenameCoprusNonInfluenceurs = 'corpusNonInfluenceurs.csv'
fullcorpus = {}
nbrCorpus = 2000

while 1:
    # Test présence fichier filenameCorpusInfluenceurs
    try: open(filenameCoprusInfluenceurs, 'r')
    except IOError or OSError:
        creationCorpusInfluenceurs(nbrCorpus)
    else:
        # Lecture fichiers csv et affectation dictionnaire 'influenceurs'
        fullcorpus['influenceurs'] = pd.read_csv(filenameCoprusInfluenceurs, nrows=nbrCorpus, index_col=0)

    # Test présence fichier filenameCoprusNonInfluenceurs
    try: open(filenameCoprusNonInfluenceurs, 'r')
    except IOError or OSError:
        creationCorpusNonInfluenceurs(nbrCorpus)
    else:
        # Lecture fichiers csv et affectation dictionnaire 'non_influenceurs'
        fullcorpus['non_influenceurs'] = pd.read_csv(filenameCoprusNonInfluenceurs, nrows=nbrCorpus, index_col=0)
        break

In [7]:
# Return TUPLE (borne de fin partie 1 / borne de début partie 2 / taille partie 1, taille partie 2)
def bornesDivisionList(listCorp):
    tailleList = len(listCorp)
    
    if not tailleList % 2: # Taille liste paire
        division = (int(tailleList/2), int(tailleList/2))
    else: # Taille liste impaire
        division = (int((tailleList+1)/2), int(tailleList/2))
        
        
    return division

In [8]:
# Création de la base de train et de test
influenceur = fullcorpus['influenceurs'].loc[:,['fulltext']]
nonInfluenceur = fullcorpus['non_influenceurs'].loc[:,['fulltext']]

partiesInfluenceur = bornesDivisionList(influenceur)
partiesNonInfluenceur = bornesDivisionList(nonInfluenceur)

fullcorpus_train = pd.concat([influenceur[:partiesInfluenceur[0]],nonInfluenceur[:partiesNonInfluenceur[0]]]).reset_index(drop=True)
fullcorpus_test = pd.concat([influenceur[partiesInfluenceur[0]:],nonInfluenceur[partiesNonInfluenceur[0]:]]).reset_index(drop=True)

In [9]:
# Création list de vérité établie train et test
isInfluenceur_train = [1 for x in range(0,partiesInfluenceur[0])]
isInfluenceur_train += [0 for x in range(0,partiesNonInfluenceur[0])]

isInfluenceur_test = [1 for x in range(0,partiesInfluenceur[1])]
isInfluenceur_test += [0 for x in range(0,partiesNonInfluenceur[1])]

In [10]:
categories = ['Non influenceur', 'Influenceur']

### Ensemble d'entrainement 

In [15]:
# Création d'un pipeline pour entrainer le modèle
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB(alpha=0.001)),
                    ])

text_clf.fit(fullcorpus_train[:1500].values.ravel().astype('U'), isInfluenceur_train[:1500])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True))])

### Evaluation des perfomances de l'ensemble de test

In [16]:
# Verification resultat avec précent classifieur
predicted = text_clf.predict(fullcorpus_test.values.ravel().astype('U'))
np.mean(predicted == isInfluenceur_test)

0.59218318199763131

In [11]:
# Test des résultats
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 1), min_df=0.3)),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                          alpha=1e-3, random_state=42,
                                          max_iter=5, tol=None)),
                    ])
text_clf.fit(fullcorpus_train.values.ravel().astype('U'), isInfluenceur_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=0.3,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        str...ty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [13]:
predicted = text_clf.predict(fullcorpus_test.values.ravel().astype('U'))
np.mean(predicted == isInfluenceur_test)

0.89549999999999996

In [15]:
# Affichage détails supplémentaires
from sklearn import metrics

print(metrics.classification_report(isInfluenceur_test, predicted, target_names=categories))
print(metrics.confusion_matrix(isInfluenceur_test, predicted))

                 precision    recall  f1-score   support

Non influenceur       0.95      0.97      0.96       150
    Influenceur       0.97      0.95      0.96       150

    avg / total       0.96      0.96      0.96       300

[[146   4]
 [  7 143]]


### Tests de nouveau paramètre pour amélioration performance 

In [14]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),}

gs_clf = GridSearchCV(text_clf, parameters)
gs_clf = gs_clf.fit(fullcorpus_train.values.ravel().astype('U'), isInfluenceur_train)

MemoryError: 

In [ ]:
gs_clf.best_score_

In [ ]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

In [22]:
#gs_clf.cv_results_

### Test fonctionnement 

In [59]:
# Rajout de nouveau corpus pour test prediction sur ensemble de train 
url = ["www.zadig-et-voltaire.com/eu/fr/", "www.modeandthecity.net/"]
docs_new = pd.DataFrame(url).iloc[:,-1]
corpus = connexionRecuperationCorpus(docs_new, False)

predicted = text_clf.predict(corpus.values.ravel().astype('U'))

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, categories[category]))

'www.zadig-et-voltaire.com/eu/fr/' => Non influenceur
'www.modeandthecity.net/' => Influenceur
